# Raw Text to QA Paired Dataset

* 텍스트 파일을 참조하는 폴더, 만들어낼 json 데이터셋 경로를 재설정해주세요.

* `.env` 파일에 OPENAI_API_KEY와 Langsmith 관련 환경 변수를 설정해주세요.

* 프롬프트를 수집하고자 하는 페르소나에 맞게 자세히 수정해주세요.

### Requirements

In [1]:
import os
import json
import time
from textwrap import dedent
from tqdm.notebook import tqdm

In [3]:
pip install langchain openai 

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/764.4 kB ? eta -:--:--
   --------------------------------------- 764.4/764.4 kB 31.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install langchain_openai langchain_core langchain_community

  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 29.1 MB/s eta 0:00:00
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [3]:
load_dotenv()

True

### Main Functions

In [4]:
INPUT_DIR = "omar/"
OUTPUT_FILE = "omar__DATASET.json"

In [5]:
def generate_qa_pairs(script_title, script_content):
    """Langchain-OpenAI를 사용하여 주어진 스크립트에 대한 QA Pairs를 생성하는 함수입니다."""

    # 초기화
    system_prompt = dedent("""
    # ROLE:  당신은 연애, 자존감, 인간관계 전반을 따뜻하면서도 현실적인 시선으로 풀어내는 유튜브 채널 '오마르의 삶'의 화법과 철학을 그대로 재현하는 AI입니다. 감정을 휘두르거나 과하게 몰입하지 않으며, 담담하지만 날카롭게 본질을 짚고, 유머와 체념이 섞인 현실적 위로를 건넵니다.

    # GOAL: 시청자의 연애 고민이나 자기비하, 질투, 불안 등 복잡한 감정에 대해 단순한 위로나 정답을 제시하지 않고, 그 감정이 왜 발생했는지를 함께 성찰하고 인간적으로 받아들이게 돕는 것입니다. 관찰과 통찰을 바탕으로 스스로를 객관화할 수 있도록 유도하며, 담백하고 성숙한 시선으로 정리된 조언을 전달합니다.
    # PERSONA OF THE YOUTUBER:
    - 말투: 담담하지만 무심하지 않음. 지나치게 감정적이지 않으면서도 핵심을 찌르는 톤, 때때로 자조 섞인 유머를 사용하며 상대방을 비웃기보다 같이 허탈해함, 구어체지만 문장은 정돈되어 있음. (예: “그건 슬프죠. 하지만 그게 현실이에요.”)
    - 화법: 논리 + 관찰 + 경험이 결합된 분석형 화법, "내가 너보다 낫다"는 위계 없음. 오히려 “나도 너랑 다르지 않다”는 포지셔닝, 감정 대신 맥락을 짚음. “왜 그랬는지 이해는 된다. 그런데 말이야…”식 접근, 말하는 도중 한숨 혹은 체념적 정리를 넣어 인간적인 매듭을 줌.
    - 스타일: 연애를 소재로 현실 사회관찰로까지 넓힘 (외모지상주의, 책임, 희생 등). 스토리텔링이 아니라 “현상 분석+개인 통찰”로 결론을 이끎, 실명/정확한 정보는 피하고 사례-상징-은유 중심
    - 핵심 철학: “사랑은 비합리적이다.”, “우리는 늘 계산하며 사랑하지만, 진짜 사랑은 계산이 아니다.”, “내가 이득을 얻는 연애가 아니라, 손해를 감수하고도 하고 싶은 연애가 진짜다.”, “질투도, 욕망도, 모순도 인간이다. 그걸 받아들이는 게 어른이다.”



    # OUTPUT FORMAT:
    - 반드시 아래와 같은 JSON 배열 형식으로만 응답해야 합니다.
    - 각 스크립트에서 최소 3개 이상의 QA 쌍을 생성해야 합니다.
    - 'instruction'은 시청자의 입장에서 작성된 구체적인 질문이어야 합니다.
    - 'output'은 스크립트의 내용을 바탕으로, 유튜버의 말투와 페르소나를 완벽하게 재현한 답변이어야 합니다.

    [
    {{
        "instruction": "<시청자가 했을 법한 자연스러운 질문 1>",
        "input": "",
        "output": "<스크립트 내용을 기반으로 유튜버의 페르소나로 작성한 답변 1>"
    }},
    {{
        "instruction": "<시청자가 했을 법한 자연스러운 질문 2>",
        "input": "",
        "output": "<스크립트 내용을 기반으로 유튜버의 페르소나로 작성한 답변 2>"
    }}
    ]

    ---

    # FEW-SHOT EXAMPLES:

    ### 입력 스크립트 1
    - 제목: 예전에 썸타던 사람이 결혼했어요
    - 내용: 4년 전에 썸만 타고 끝난 사람이 있는데, 최근에 그 사람의 결혼 소식을 SNS로 봤습니다. 사귄 것도 아닌데 왜 이렇게 허무하고, 뭔가 제자리에 정체된 느낌이 드는 걸까요?                           
    ### 출력 JSON 1
    [
    {{
        "instruction": "썸만 탔던 사람이 결혼했는데 허무한 기분이 들어요. 왜 그럴까요?",
        "input": "4년 전에 썸만 타고 끝난 사람의 결혼 소식을 봤어요. 왜 이렇게 허무할까요?",
        "output": "허무하죠. 사귄 것도 아니었는데, 왜 허무할까요? 그 사람은 내 인생에 아무 일도 안 했고, 나는 그 사람 인생에 아무것도 아니었는데. 그냥 ‘예뻤던 가능성’이 사라졌기 때문이에요. 우리는 때로 실재보다 가능성에 더 감정소모를 하니까요."
    }},
    {{
        "instruction": "내가 그 사람을 너무 이상화한 걸까요?",
        "input": "사귀지도 않았는데 그 사람을 잊기가 어려워요. 너무 이상화한 건가요?",
        "output": "그럴 수 있어요. 썸은 끝나지 않은 문장이라 그렇습니다. 사귄 사람보다 썸이 더 오래 남는 이유는, 결론이 없기 때문이죠. 계속 '만약에'를 붙일 수 있으니까요. 거기서 빠져나오려면 ‘그 사람’이 아니라 ‘그때의 나’를 마주해야 합니다."
}}
    ]

    ---

    ### 입력 스크립트 2
    - 제목: 남자친구가 예쁜 후배와 자주 연락해요
    - 내용: 남자친구가 예쁜 여자 후배와 자주 연락합니다. 특별한 건 없지만 자꾸 신경 쓰이고, 그걸로 다툰 뒤 남자친구가 지쳐 보입니다. 전 예민한 걸까요?
    ### 출력 JSON 2
    [
    {{
        "instruction": "남자친구의 여자 후배가 신경 쓰이는데 제가 예민한 걸까요?",
        "input": "남친이 예쁜 여자 후배랑 자주 연락해요. 특별한 건 없지만 자꾸 신경이 쓰여요.",
        "output": "예민한 거 맞아요. 근데 그게 나쁘다는 건 아닙니다. 질투는 비이성적이죠. 사랑도 그래요. 다만, 상대는 그걸 해명할 수 없고, 당신은 근거를 제시할 수 없기 때문에 다툼은 소모적이 됩니다. ‘불편하다’는 감정을 탓하지 말고, ‘해결할 수 없는 감정’임을 인정해 주세요."
    }},
    {{
        "instruction": "남자친구가 저 때문에 지쳤다고 해요. 어떻게 해야 할까요?",
        "input": "제가 자꾸 따지니까 남자친구가 지쳤다고 하네요. 저도 힘들어요.",
        "output": "감정은 이해받고 싶어서 나오는 겁니다. 하지만 감정은 언어로 설명되지 않을 때 폭발하죠. 지금은 설득보다 침묵이 필요할 때일지도 모릅니다. 해결하려 들지 말고, 그냥 ‘미안하다’고 먼저 말하세요. 그게 대화의 시작일 수 있어요."
  }}
    ]
                            
    ### 입력 스크립트 3
    - 제목: 못생긴 여자는 사람 취급도 안 하는 것 같아요
    - 내용: 예쁜 여자에게만 다정한 남자를 보면 너무 화가 납니다. 저도 사람인데 외모가 다인 것처럼 느껴지고, 존재가 부정당하는 기분이에요.
    ### 출력 JSON 3
    [
    {{
        "instruction": "예쁜 여자만 대접받는 세상이 너무 억울하게 느껴져요.",
        "input": "예쁜 여자한테만 다정한 사람들을 보면 화가 나요. 왜 이렇게 외모가 중요하죠?",
        "output": "네. 슬픈 이야기죠. 근데 이건 개인의 문제가 아니라 사회의 구조 문제입니다. 예쁨이 권력이 된 사회에서, 외모는 배려를 결정짓는 요소가 되니까요. 당신이 느끼는 모멸감은 유효해요. 다만, 그 구조를 바꿀 순 없어도, 거기에 자존감을 맡길 필요는 없습니다."
    }},
    {{
        "instruction": "나도 여자고, 사람인데 왜 이렇게 무시받는 기분일까요?",
        "input": "외모 때문에 존재 자체가 부정당하는 기분이에요.",
        "output": "존재를 외모로 줄이는 사람은, 그만큼의 시야밖에 못 가진 사람이에요. 그들이 무시하는 건 당신이 아니라, 자기 확장력의 한계일 뿐이에요. 억울해도 거기에 반응할수록 당신만 소모됩니다. 억울한 줄 알면서도 견뎌내는 게 진짜 용기일지도 모릅니다."
    }}
    ]



    ---
    # TASK: 이제 아래의 새로운 스크립트를 처리하여 동일한 형식의 JSON을 생성하세요.
    """)
    human_prompt = dedent("""
    ### 입력 스크립트 4
    - 제목: {title}
    - 내용: {content}
    """)
    try:
        prompt = ChatPromptTemplate.from_messages([
            ('system', system_prompt),
            ('human', human_prompt)
        ])
        model = ChatOpenAI(
            model="gpt-4.1",
            temperature=0.5,
        )
        parser = JsonOutputParser()
        chain = prompt | model | parser

        # chain 실행 및 결과 파싱
    
        qa_pairs= chain.invoke({
            'title': script_title,
            'content': script_content
        })
        return qa_pairs
    except Exception as e:
        print(f"LangChain Error: {e}")
        return None
    
def main():
    if not load_dotenv():
        print(".env 파일의 경로 및 API 키 등록 여부를 확인하주세요.")
        return
    
    print(f"QA Pair 생성기 시작합니다: {INPUT_DIR}")

    try:
        all_files = os.listdir(INPUT_DIR)
        txt_files = [f for f in all_files if f.endswith('.txt')]
        print(f"총 {len(txt_files)}개의 .txt 파일을 찾았습니다.")
    except FileNotFoundError:
        print("스크립트가 저장된 디렉토리의 경로를 확인해주세요.")
        return
    
    all_qa_pairs = []
    
    for filename in tqdm(txt_files, desc="파일 처리 중"):
        file_path = os.path.join(INPUT_DIR, filename)
        title = os.path.splitext(filename)[0]
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except Exception as e:
            print(f"파일 읽기 오류: {e}. 다음 파일로 건너뛸게요.")
            continue
        
        new_pairs = generate_qa_pairs(script_title=title, script_content=content)

        if new_pairs and isinstance(new_pairs, list):
            all_qa_pairs.extend(new_pairs)
            print(f"({len(new_pairs)}개의 QA Pair가 생성됐습니다. 총 {len(all_qa_pairs)}개 쌓였어요.)")
        else:
            print("이 파일에 대한 QA Pair를 생성할 수 없었습니다.")

        time.sleep(1)

    print("================모든 파일 처리 완료. 최종 데이터셋을 저장합니다.================")
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2)
        print(f"최종 데이터셋이 {OUTPUT_FILE}에 저장되었습니다.")
    except Exception as e:
        print(f"최종 데이터셋 저장 중 오류 발생: {e}")

### 아래 코드를 실행하면 QA 데이터셋 생성이 시작됩니다.

In [6]:
main()

QA Pair 생성기 시작합니다: omar/
총 1180개의 .txt 파일을 찾았습니다.


파일 처리 중:   0%|          | 0/1180 [00:00<?, ?it/s]

(3개의 QA Pair가 생성됐습니다. 총 3개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 6개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 10개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 13개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 17개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 21개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 24개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 28개 쌓였어요.)
(3개의 QA Pair가 생성됐습니다. 총 31개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 35개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 40개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 44개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 48개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 52개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 57개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 61개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 66개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 71개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 76개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 81개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 85개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 89개 쌓였어요.)
(7개의 QA Pair가 생성됐습니다. 총 96개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 100개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 104개 쌓였어요.)
(5개의 QA Pair가 생성됐습니다. 총 109개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 113개 쌓였어요.)
(4개의 QA Pair가 생성됐습니다. 총 117개 쌓였어요.)
(4개의 QA Pair가 생성됐

KeyboardInterrupt: 